<a href="https://colab.research.google.com/github/varunaluri18/pyspark/blob/main/pyspark_structured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 41.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=c59f25c0488784296ef4cee4aff6d6f3c405aa69ad6e6b12b96a2d3efaa5eb88
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark =SparkSession.builder.appName('firstprogram').getOrCreate()
spark

In [ ]:
data=[('varun','nagendra','M',28000),('sravani','yakkanti','F',27000),('sri','vidya','NA',26000),('ishwarya','himabindu','F',26500),(None,'kaveri','F',24900)]
columns=["firstname","lastname","gender","salary"]
df=spark.createDataFrame(data=data, schema = columns)
df.show(truncate=False)

+---------+---------+------+------+
|firstname|lastname |gender|salary|
+---------+---------+------+------+
|varun    |nagendra |M     |28000 |
|sravani  |yakkanti |F     |27000 |
|sri      |vidya    |NA    |26000 |
|ishwarya |himabindu|F     |26500 |
|null     |kaveri   |F     |24900 |
+---------+---------+------+------+



In [ ]:
print(df.collect())

[Row(firstname='varun', lastname='nagendra', gender='M', salary=28000), Row(firstname='sravani', lastname='yakkanti', gender='F', salary=27000), Row(firstname='sri', lastname='vidya', gender='NA', salary=26000), Row(firstname='ishwarya', lastname='himabindu', gender='F', salary=26500), Row(firstname=None, lastname='kaveri', gender='F', salary=24900)]


In [ ]:
df2=df.withColumn("salary", df.salary*3)
df2.show()

+---------+---------+------+------+
|firstname| lastname|gender|salary|
+---------+---------+------+------+
|    varun| nagendra|     M| 84000|
|  sravani| yakkanti|     F| 81000|
|      sri|    vidya|    NA| 78000|
| ishwarya|himabindu|     F| 79500|
|     null|   kaveri|     F| 74700|
+---------+---------+------+------+



In [ ]:
from pyspark.sql.functions import when
df3 = df.withColumn("gender", when(df.gender == "M","Male").when(df.gender == "F","Female").otherwise(df.gender))
df3.show()

+---------+---------+------+------+
|firstname| lastname|gender|salary|
+---------+---------+------+------+
|    varun| nagendra|  Male| 28000|
|  sravani| yakkanti|Female| 27000|
|      sri|    vidya|    NA| 26000|
| ishwarya|himabindu|Female| 26500|
|     null|   kaveri|Female| 24900|
+---------+---------+------+------+



In [ ]:
df4=df.withColumn("salary",df.salary.cast("String"))
df4.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)



In [ ]:
df.createOrReplaceTempView("PER")
df5=spark.sql("select firstname,gender,salary*3 as salary from PER")
df5.show()

+---------+------+------+
|firstname|gender|salary|
+---------+------+------+
|    varun|     M| 84000|
|  sravani|     F| 81000|
|      sri|    NA| 78000|
| ishwarya|     F| 79500|
|     null|     F| 74700|
+---------+------+------+



In [ ]:
# Pandas-PySpark-DataFrame

In [ ]:
import pandas as pd    
data = [['varun', 24], ['sravani', 23], ['ishwarya', 27],['vidya', 25]]   
pandasDF = pd.DataFrame(data, columns = ['Name', 'Age']) 
print(pandasDF,type(pandasDF))

       Name  Age
0     varun   24
1   sravani   23
2  ishwarya   27
3     vidya   25 <class 'pandas.core.frame.DataFrame'>


In [ ]:
sparkDF=spark.createDataFrame(pandasDF) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)

+--------+---+
|    Name|Age|
+--------+---+
|   varun| 24|
| sravani| 23|
|ishwarya| 27|
|   vidya| 25|
+--------+---+



In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
mySchema = StructType([ StructField("First Name", StringType(), True),StructField("Age", IntegerType(), True)])

In [ ]:
sparkDF2 = spark.createDataFrame(pandasDF,schema=mySchema)
sparkDF2.printSchema()
sparkDF2.show()

root
 |-- First Name: string (nullable = true)
 |-- Age: integer (nullable = true)

+----------+---+
|First Name|Age|
+----------+---+
|     varun| 24|
|   sravani| 23|
|  ishwarya| 27|
|     vidya| 25|
+----------+---+



In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")
spark.conf.set("spark.sql.execution.arrow.pyspark.fallback.enabled","true")

In [ ]:
# adding months

In [ ]:
from pyspark.sql.functions import col,expr
data=[("2019-01-23",1),("2019-06-24",2),("2019-09-20",3)]
spark.createDataFrame(data).toDF("date","increment").select(col("date"),col("increment"),expr("add_months(to_date(date,'yyyy-MM-dd'),cast(increment as int))").alias("inc_date")).show()

+----------+---------+----------+
|      date|increment|  inc_date|
+----------+---------+----------+
|2019-01-23|        1|2019-02-23|
|2019-06-24|        2|2019-08-24|
|2019-09-20|        3|2019-12-20|
+----------+---------+----------+



In [ ]:
# adding new coloumn

In [ ]:
data=[('varun','nagendra','M',28000),('sravani','yakkanti','F',27000),('sri','vidya','NA',26000),('ishwarya','himabindu','F',26500),('kaveri','rayi','F',24900)]
columns=["firstname","lastname","gender","salary"]
df=spark.createDataFrame(data=data, schema = columns)
df.show(truncate=False)
if 'age' not in df.columns:
    print("No age was defined in the data frame")

+---------+---------+------+------+
|firstname|lastname |gender|salary|
+---------+---------+------+------+
|varun    |nagendra |M     |28000 |
|sravani  |yakkanti |F     |27000 |
|sri      |vidya    |NA    |26000 |
|ishwarya |himabindu|F     |26500 |
|kaveri   |rayi     |F     |24900 |
+---------+---------+------+------+

No age was defined in the data frame


In [ ]:
from pyspark.sql.functions import lit
print('Adding the new coloumn using lit function')
df.withColumn("bonus_percent", lit(0.3)).show()

print('Add column from existing column')
df.withColumn("bonus_amount", df.salary*0.3).show()

print('Add column by concatinating existing column using concat_ws')
from pyspark.sql.functions import concat_ws
df.withColumn("name", concat_ws(" ","firstname",'lastname')).show()

print("Add current date")
from pyspark.sql.functions import current_date
df.withColumn("current_date", current_date()).show()

Adding the new coloumn using lit function
+---------+---------+------+------+-------------+
|firstname| lastname|gender|salary|bonus_percent|
+---------+---------+------+------+-------------+
|    varun| nagendra|     M| 28000|          0.3|
|  sravani| yakkanti|     F| 27000|          0.3|
|      sri|    vidya|    NA| 26000|          0.3|
| ishwarya|himabindu|     F| 26500|          0.3|
|   kaveri|     rayi|     F| 24900|          0.3|
+---------+---------+------+------+-------------+

Add column from existing column
+---------+---------+------+------+------------+
|firstname| lastname|gender|salary|bonus_amount|
+---------+---------+------+------+------------+
|    varun| nagendra|     M| 28000|      8400.0|
|  sravani| yakkanti|     F| 27000|      8100.0|
|      sri|    vidya|    NA| 26000|      7800.0|
| ishwarya|himabindu|     F| 26500|      7950.0|
|   kaveri|     rayi|     F| 24900|      7470.0|
+---------+---------+------+------+------------+

Add column by concatinating exist

In [ ]:
from pyspark.sql.functions import when
from pyspark.sql.functions import lit
print('Using the lit and when function')
df.withColumn("grade",when((df.salary >=27000), lit("A")).when((df.salary <27000) & (df.salary >=25000), lit("B")).otherwise(lit("C"))).show()

Using the lit and when function
+---------+---------+------+------+-----+
|firstname| lastname|gender|salary|grade|
+---------+---------+------+------+-----+
|    varun| nagendra|     M| 28000|    A|
|  sravani| yakkanti|     F| 27000|    A|
|      sri|    vidya|    NA| 26000|    B|
| ishwarya|himabindu|     F| 26500|    B|
|   kaveri|     rayi|     F| 24900|    C|
+---------+---------+------+------+-----+



In [ ]:
print('Adding coloumn using select')
from pyspark.sql.functions import lit
df.select("firstname","salary", lit(0.3).alias("bonus")).show()
df.select("firstname","salary", lit(df.salary * 0.3).alias("bonus_amount")).show()
df.select("firstname","salary", current_date().alias("today_date")).show()

Adding coloumn using select
+---------+------+-----+
|firstname|salary|bonus|
+---------+------+-----+
|    varun| 28000|  0.3|
|  sravani| 27000|  0.3|
|      sri| 26000|  0.3|
| ishwarya| 26500|  0.3|
|   kaveri| 24900|  0.3|
+---------+------+-----+

+---------+------+------------+
|firstname|salary|bonus_amount|
+---------+------+------------+
|    varun| 28000|      8400.0|
|  sravani| 27000|      8100.0|
|      sri| 26000|      7800.0|
| ishwarya| 26500|      7950.0|
|   kaveri| 24900|      7470.0|
+---------+------+------------+

+---------+------+----------+
|firstname|salary|today_date|
+---------+------+----------+
|    varun| 28000|2021-11-18|
|  sravani| 27000|2021-11-18|
|      sri| 26000|2021-11-18|
| ishwarya| 26500|2021-11-18|
|   kaveri| 24900|2021-11-18|
+---------+------+----------+



In [ ]:
print('Ading coloumns using Spark SQL')
df.createOrReplaceTempView("temp")
spark.sql("select firstname,salary, '0.3' as bonus from temp").show()
spark.sql("select firstname,salary, salary * 0.3 as bonus_amount from temp").show()
spark.sql("select firstname,salary, current_date() as today_date from temp").show()

print('Using CASE statement in SQL')
spark.sql("select firstname,salary, " +"case salary when salary > 25000 then 'B' "+"else 'B' END as grade from temp").show()

Ading coloumns using Spark SQL
+---------+------+-----+
|firstname|salary|bonus|
+---------+------+-----+
|    varun| 28000|  0.3|
|  sravani| 27000|  0.3|
|      sri| 26000|  0.3|
| ishwarya| 26500|  0.3|
|   kaveri| 24900|  0.3|
+---------+------+-----+

+---------+------+------------+
|firstname|salary|bonus_amount|
+---------+------+------------+
|    varun| 28000|      8400.0|
|  sravani| 27000|      8100.0|
|      sri| 26000|      7800.0|
| ishwarya| 26500|      7950.0|
|   kaveri| 24900|      7470.0|
+---------+------+------------+

+---------+------+----------+
|firstname|salary|today_date|
+---------+------+----------+
|    varun| 28000|2021-11-18|
|  sravani| 27000|2021-11-18|
|      sri| 26000|2021-11-18|
| ishwarya| 26500|2021-11-18|
|   kaveri| 24900|2021-11-18|
+---------+------+----------+

Using CASE statement in SQL
+---------+------+-----+
|firstname|salary|grade|
+---------+------+-----+
|    varun| 28000|    B|
|  sravani| 27000|    B|
|      sri| 26000|    B|
| ish

In [ ]:
print("Applying Aggrigate Functions")
from pyspark.sql.functions import approx_count_distinct,collect_list
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count
from pyspark.sql.functions import first, last, kurtosis, min, mean, skewness 
from pyspark.sql.functions import stddev, stddev_samp, stddev_pop, sumDistinct
from pyspark.sql.functions import variance,var_samp,  var_pop

Applying Aggrigate Functions


In [ ]:
print("Distinct Salaries: " +str(df.select(approx_count_distinct("salary")).collect()[0][0]))

Distinct Salaries: 5


In [ ]:
df.select(countDistinct("firstname")).show()

+-------------------------+
|count(DISTINCT firstname)|
+-------------------------+
|                        5|
+-------------------------+



In [ ]:
print("Average salary of all the employees: " + str(df.select(avg("salary")).collect()[0][0]))

Average salary of all the employees: 26480.0


In [ ]:
df.select(collect_list("salary")).show(truncate=False)

+-----------------------------------+
|collect_list(salary)               |
+-----------------------------------+
|[28000, 27000, 26000, 26500, 24900]|
+-----------------------------------+



In [ ]:
df.select(collect_set("salary")).show(truncate=False)

+-----------------------------------+
|collect_set(salary)                |
+-----------------------------------+
|[26000, 24900, 26500, 27000, 28000]|
+-----------------------------------+



In [ ]:
simpleData = [("James", "Sales", 3000),("Michael", "Sales", 4600),("Robert", "Sales", 4100),("Maria", "Finance", 3000),("James", "Sales", 3000),("Scott", "Finance", 3300),("Jen", "Finance", 3900),("Jeff", "Marketing", 3000),("Kumar", "Marketing", 2000),("Saif", "Sales", 4100)]
schema = ["employee_name", "department", "salary"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [ ]:
df2 = df.select(countDistinct("department", "salary"))
df2.show(truncate=False)
print("Distinct Count of Department &amp; Salary: "+str(df2.collect()[0][0]))

+----------------------------------+
|count(DISTINCT department, salary)|
+----------------------------------+
|8                                 |
+----------------------------------+

Distinct Count of Department &amp; Salary: 8


In [ ]:
print("count: "+str(df.select(count("department")).collect()[0][0]))

count: 10


In [ ]:
print('Salary of the first person on the top')
df.select(first("salary")).show(truncate=False)

print('Salary of the last person on the bottom')
df.select(last("salary")).show(truncate=False)

print('Finding Kurtosis')
df.select(kurtosis("salary")).show(truncate=False)

print('Finding Skewness')
df.select(skewness("salary")).show(truncate=False)

print('Finding Maximum')
df.select(max("salary")).show(truncate=False)

print('Finding Minimum')
df.select(min("salary")).show(truncate=False)

print('Finding Average')
df.select(mean("salary")).show(truncate=False)

print('Finding the sum of distinct salaries')
df.select(sumDistinct("salary")).show(truncate=False)

print('Finding standard deviation')
df.select(stddev("salary"), stddev_samp("salary"),stddev_pop("salary")).show(truncate=False)

print('Finding varience')
df.select(variance("salary"),var_samp("salary"),var_pop("salary")).show(truncate=False)

Salary of the first person on the top
+-------------+
|first(salary)|
+-------------+
|3000         |
+-------------+

Salary of the last person on the bottom
+------------+
|last(salary)|
+------------+
|4100        |
+------------+

Finding Kurtosis
+-------------------+
|kurtosis(salary)   |
+-------------------+
|-0.6467803030303032|
+-------------------+

Finding Skewness
+--------------------+
|skewness(salary)    |
+--------------------+
|-0.12041791181069571|
+--------------------+

Finding Maximum
+-----------+
|max(salary)|
+-----------+
|4600       |
+-----------+

Finding Minimum
+-----------+
|min(salary)|
+-----------+
|2000       |
+-----------+

Finding Average
+-----------+
|avg(salary)|
+-----------+
|3400.0     |
+-----------+

Finding the sum of distinct salaries


/usr/local/lib/python3.7/dist-packages/pyspark/sql/functions.py:214: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+--------------------+
|sum(DISTINCT salary)|
+--------------------+
|20900               |
+--------------------+

Finding standard deviation
+-------------------+-------------------+------------------+
|stddev_samp(salary)|stddev_samp(salary)|stddev_pop(salary)|
+-------------------+-------------------+------------------+
|765.9416862050705  |765.9416862050705  |726.636084983398  |
+-------------------+-------------------+------------------+

Finding varience
+-----------------+-----------------+---------------+
|var_samp(salary) |var_samp(salary) |var_pop(salary)|
+-----------------+-----------------+---------------+
|586666.6666666666|586666.6666666666|528000.0       |
+-----------------+-----------------+---------------+



In [ ]:
print('PySpark Array String')

PySpark Array String


In [ ]:
columns = ["name","languagesAtSchool","currentState"]
data = [("James,Smith",["Java","Scala","C++"],"CA"),("Michael,Rose,",["Spark","Java","C++"],"NJ"),("Robert,Williams",["CSharp","VB"],"NV")]
df = spark.createDataFrame(data=data,schema=columns)
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- languagesAtSchool: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- currentState: string (nullable = true)

+---------------+------------------+------------+
|           name| languagesAtSchool|currentState|
+---------------+------------------+------------+
|    James,Smith|[Java, Scala, C++]|          CA|
|  Michael,Rose,|[Spark, Java, C++]|          NJ|
|Robert,Williams|      [CSharp, VB]|          NV|
+---------------+------------------+------------+



In [ ]:
from pyspark.sql.functions import col, concat_ws
df2 = df.withColumn("languagesAtSchool",concat_ws(",",col("languagesAtSchool")))
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- languagesAtSchool: string (nullable = false)
 |-- currentState: string (nullable = true)

+---------------+-----------------+------------+
|name           |languagesAtSchool|currentState|
+---------------+-----------------+------------+
|James,Smith    |Java,Scala,C++   |CA          |
|Michael,Rose,  |Spark,Java,C++   |NJ          |
|Robert,Williams|CSharp,VB        |NV          |
+---------------+-----------------+------------+



In [ ]:
df.createOrReplaceTempView("arr")
spark.sql("select name, concat_ws(',',languagesAtSchool) as languagesAtSchool," + " currentState from arr").show(truncate=False)

+---------------+-----------------+------------+
|name           |languagesAtSchool|currentState|
+---------------+-----------------+------------+
|James,Smith    |Java,Scala,C++   |CA          |
|Michael,Rose,  |Spark,Java,C++   |NJ          |
|Robert,Williams|CSharp,VB        |NV          |
+---------------+-----------------+------------+



In [ ]:
print('changing the data types of the coloumns')

changing the data types of the coloumns


In [ ]:
simpleData = [("varun",34,"2006-01-01","true","M",3000.60),("sravani",33,"1980-01-10","true","F",3300.80),("vidya",37,"06-01-1992","false","F",5000.50)]
columns = ["firstname","age","jobStartDate","isGraduated","gender","salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

changing the data types of the coloumns
root
 |-- firstname: string (nullable = true)
 |-- age: long (nullable = true)
 |-- jobStartDate: string (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)

+---------+---+------------+-----------+------+------+
|firstname|age|jobStartDate|isGraduated|gender|salary|
+---------+---+------------+-----------+------+------+
|varun    |34 |2006-01-01  |true       |M     |3000.6|
|sravani  |33 |1980-01-10  |true       |F     |3300.8|
|vidya    |37 |06-01-1992  |false      |F     |5000.5|
+---------+---+------------+-----------+------+------+



In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType
df2 = df.withColumn("age",col("age").cast(StringType())).withColumn("isGraduated",col("isGraduated").cast(BooleanType())).withColumn("jobStartDate",col("jobStartDate").cast(DateType()))
df2.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- age: string (nullable = true)
 |-- jobStartDate: date (nullable = true)
 |-- isGraduated: boolean (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)



In [ ]:
df3 = df2.selectExpr("cast(age as int) age","cast(isGraduated as string) isGraduated","cast(jobStartDate as string) jobStartDate")
df3.printSchema()
df3.show(truncate=False)

root
 |-- age: integer (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- jobStartDate: string (nullable = true)

+---+-----------+------------+
|age|isGraduated|jobStartDate|
+---+-----------+------------+
|34 |true       |2006-01-01  |
|33 |true       |1980-01-10  |
|37 |false      |null        |
+---+-----------+------------+



In [ ]:
df3.createOrReplaceTempView("Cast")
df4 = spark.sql("SELECT STRING(age),BOOLEAN(isGraduated),DATE(jobStartDate) from Cast")
df4.printSchema()
df4.show(truncate=False)

root
 |-- age: string (nullable = true)
 |-- isGraduated: boolean (nullable = true)
 |-- jobStartDate: date (nullable = true)

+---+-----------+------------+
|age|isGraduated|jobStartDate|
+---+-----------+------------+
|34 |true       |2006-01-01  |
|33 |true       |1980-01-10  |
|37 |false      |null        |
+---+-----------+------------+



In [ ]:
from pyspark.sql.functions import col,round,expr
df.withColumn("salary",df.salary.cast('float')).printSchema()    
df.withColumn("salary",col("salary").cast('double')).printSchema()

root
 |-- firstname: string (nullable = true)
 |-- age: long (nullable = true)
 |-- jobStartDate: string (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: float (nullable = true)

root
 |-- firstname: string (nullable = true)
 |-- age: long (nullable = true)
 |-- jobStartDate: string (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)



In [ ]:
df.selectExpr("firstname","isGraduated","cast(salary as double) salary").printSchema()    

root
 |-- firstname: string (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- salary: double (nullable = true)



In [ ]:
df.createOrReplaceTempView("cast")
spark.sql("SELECT firstname,isGraduated,DOUBLE(salary) as salary from cast").printSchema()

root
 |-- firstname: string (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- salary: double (nullable = true)



In [ ]:
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.show(truncate=False)

data = deptDF.collect()
print(data)
print()
dataCollect2 = deptDF.select("dept_name").collect()
print(dataCollect2)
print()
for row in data:
    print(row['dept_name'] + "," +str(row['dept_id']))

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+

[Row(dept_name='Finance', dept_id=10), Row(dept_name='Marketing', dept_id=20), Row(dept_name='Sales', dept_id=30), Row(dept_name='IT', dept_id=40)]

[Row(dept_name='Finance'), Row(dept_name='Marketing'), Row(dept_name='Sales'), Row(dept_name='IT')]

Finance,10
Marketing,20
Sales,30
IT,40


In [ ]:
print('PySpark Coloumn Functions')

PySpark Coloumn Functions


In [ ]:
data=[("varun","nagendra","100",'M'),("sravani","yakkanti","200",'F'),("sri","vidya","400",'F'),("ishwarya","himabindu","400",'F')] 
columns=["fname","lname","id","gender"]
df=spark.createDataFrame(data,columns)
print(df.show(truncate=False))

+--------+---------+---+------+
|fname   |lname    |id |gender|
+--------+---------+---+------+
|varun   |nagendra |100|M     |
|sravani |yakkanti |200|F     |
|sri     |vidya    |400|F     |
|ishwarya|himabindu|400|F     |
+--------+---------+---+------+

None


In [ ]:
#alias
from pyspark.sql.functions import expr
df.select(df.fname.alias("first_name"),df.lname.alias("last_name"),expr(" fname ||','|| lname").alias("fullName")).show()

+----------+---------+------------------+
|first_name|last_name|          fullName|
+----------+---------+------------------+
|     varun| nagendra|    varun,nagendra|
|   sravani| yakkanti|  sravani,yakkanti|
|       sri|    vidya|         sri,vidya|
|  ishwarya|himabindu|ishwarya,himabindu|
+----------+---------+------------------+



In [ ]:
#asc, desc
df.sort(df.id.asc()).show()
df.sort(df.id.desc()).show()

+--------+---------+---+------+
|   fname|    lname| id|gender|
+--------+---------+---+------+
|   varun| nagendra|100|     M|
| sravani| yakkanti|200|     F|
|     sri|    vidya|400|     F|
|ishwarya|himabindu|400|     F|
+--------+---------+---+------+

+--------+---------+---+------+
|   fname|    lname| id|gender|
+--------+---------+---+------+
|     sri|    vidya|400|     F|
|ishwarya|himabindu|400|     F|
| sravani| yakkanti|200|     F|
|   varun| nagendra|100|     M|
+--------+---------+---+------+



In [ ]:
#cast
df.select(df.fname,df.id.cast("int")).show()

+--------+---+
|   fname| id|
+--------+---+
|   varun|100|
| sravani|200|
|     sri|400|
|ishwarya|400|
+--------+---+



In [ ]:
#between
df.filter(df.id.between(100,300)).show()

+-------+--------+---+------+
|  fname|   lname| id|gender|
+-------+--------+---+------+
|  varun|nagendra|100|     M|
|sravani|yakkanti|200|     F|
+-------+--------+---+------+



In [ ]:
#contains
df.filter(df.fname.contains("varun")).show()

+-----+--------+---+------+
|fname|   lname| id|gender|
+-----+--------+---+------+
|varun|nagendra|100|     M|
+-----+--------+---+------+



In [ ]:
#startswith, endswith()
df.filter(df.fname.startswith("v")).show()
df.filter(df.fname.endswith("i")).show()

+-----+--------+---+------+
|fname|   lname| id|gender|
+-----+--------+---+------+
|varun|nagendra|100|     M|
+-----+--------+---+------+

+-------+--------+---+------+
|  fname|   lname| id|gender|
+-------+--------+---+------+
|sravani|yakkanti|200|     F|
|    sri|   vidya|400|     F|
+-------+--------+---+------+



In [ ]:
#eqNullSafe
#isNull & isNotNull
df.filter(df.lname.isNull()).show()
df.filter(df.lname.isNotNull()).show()

+-----+-----+---+------+
|fname|lname| id|gender|
+-----+-----+---+------+
+-----+-----+---+------+

+--------+---------+---+------+
|   fname|    lname| id|gender|
+--------+---------+---+------+
|   varun| nagendra|100|     M|
| sravani| yakkanti|200|     F|
|     sri|    vidya|400|     F|
|ishwarya|himabindu|400|     F|
+--------+---------+---+------+



In [ ]:
#like , rlike
df.select(df.fname,df.lname,df.id).filter(df.fname.like("%varun")).show() 

+-----+--------+---+
|fname|   lname| id|
+-----+--------+---+
|varun|nagendra|100|
+-----+--------+---+



In [ ]:
#over ,#substr
df.select(df.fname.substr(1,4).alias("substr")).show()

+------+
|substr|
+------+
|  varu|
|  srav|
|   sri|
|  ishw|
+------+



In [ ]:
#when & otherwise
from pyspark.sql.functions import when
df.select(df.fname,df.lname,when(df.gender=="M","Male").when(df.gender=="F","Female") .when(df.gender==None ,"").otherwise(df.gender).alias("new_gender")).show()

+--------+---------+----------+
|   fname|    lname|new_gender|
+--------+---------+----------+
|   varun| nagendra|      Male|
| sravani| yakkanti|    Female|
|     sri|    vidya|    Female|
|ishwarya|himabindu|    Female|
+--------+---------+----------+



In [ ]:
#isin
li=["100","200"]
df.select(df.fname,df.lname,df.id).filter(df.id.isin(li)).show()

+-------+--------+---+
|  fname|   lname| id|
+-------+--------+---+
|  varun|nagendra|100|
|sravani|yakkanti|200|
+-------+--------+---+



In [ ]:
from pyspark.sql.types import StructType,StructField,StringType,ArrayType,MapType
data=[(("James","Bond"),["Java","C#"],{'hair':'black','eye':'brown'}),(("Ann","Varsa"),[".NET","Python"],{'hair':'brown','eye':'black'}),(("Tom Cruise",""),["Python","Scala"],{'hair':'red','eye':'grey'}),(("Tom Brand",None),["Perl","Ruby"],{'hair':'black','eye':'blue'})]
schema = StructType([StructField('name', StructType([StructField('fname', StringType(), True),StructField('lname', StringType(), True)])),StructField('languages', ArrayType(StringType()),True),StructField('properties', MapType(StringType(),StringType()),True)])


In [ ]:
df=spark.createDataFrame(data,schema)
df.printSchema()
df.show()

root
 |-- name: struct (nullable = true)
 |    |-- fname: string (nullable = true)
 |    |-- lname: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+-----------------+---------------+--------------------+
|             name|      languages|          properties|
+-----------------+---------------+--------------------+
|    {James, Bond}|     [Java, C#]|{eye -> brown, ha...|
|     {Ann, Varsa}| [.NET, Python]|{eye -> black, ha...|
|   {Tom Cruise, }|[Python, Scala]|{eye -> grey, hai...|
|{Tom Brand, null}|   [Perl, Ruby]|{eye -> blue, hai...|
+-----------------+---------------+--------------------+



In [ ]:
#getItem()
df.select(df.languages.getItem(0)).show()
df.select(df.properties.getItem("hair")).show()

+------------+
|languages[0]|
+------------+
|        Java|
|        .NET|
|      Python|
|        Perl|
+------------+

+----------------+
|properties[hair]|
+----------------+
|           black|
|           brown|
|             red|
|           black|
+----------------+



In [ ]:
#getField from Struct or Map
df.select(df.properties.getField("hair")).show()
df.select(df.name.getField("fname")).show()

+----------------+
|properties[hair]|
+----------------+
|           black|
|           brown|
|             red|
|           black|
+----------------+

+----------+
|name.fname|
+----------+
|     James|
|       Ann|
|Tom Cruise|
| Tom Brand|
+----------+



In [ ]:
#withField
from pyspark.sql.functions import lit
df.withColumn("name",df.name.withField("fname",lit("AA"))).show()

+-----------+---------------+--------------------+
|       name|      languages|          properties|
+-----------+---------------+--------------------+
| {AA, Bond}|     [Java, C#]|{eye -> brown, ha...|
|{AA, Varsa}| [.NET, Python]|{eye -> black, ha...|
|     {AA, }|[Python, Scala]|{eye -> grey, hai...|
| {AA, null}|   [Perl, Ruby]|{eye -> blue, hai...|
+-----------+---------------+--------------------+



In [ ]:
data=[("Varun",24),("Nagendra",23)]
df=spark.createDataFrame(data).toDF("name","gender")
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- gender: long (nullable = true)

+--------+------+
|    name|gender|
+--------+------+
|   Varun|    24|
|Nagendra|    23|
+--------+------+



In [ ]:
from pyspark.sql.functions import col
print('selecting coloumn from dataframe using col')
df.select(col("name")).show()
print('selecting coloumn from dataframe')
df.select(df["name"]).show()
print('adding new_coloumn using withColumn')
df.withColumn("new_col",col("name").substr(1,4)).show()
print('Applying Filters')
df.filter(col("name").startswith("V")).show()

selecting coloumn from dataframe using col
+--------+
|    name|
+--------+
|   Varun|
|Nagendra|
+--------+

selecting coloumn from dataframe
+--------+
|    name|
+--------+
|   Varun|
|Nagendra|
+--------+

adding new_coloumn using withColumn
+--------+------+-------+
|    name|gender|new_col|
+--------+------+-------+
|   Varun|    24|   Varu|
|Nagendra|    23|   Nage|
+--------+------+-------+

Applying Filters
+-----+------+
| name|gender|
+-----+------+
|Varun|    24|
+-----+------+



In [ ]:
# Using DataFrame object
df.select(df.gender).show()
df.select(df["gender"]).show()

+------+
|gender|
+------+
|    24|
|    23|
+------+

+------+
|gender|
+------+
|    24|
|    23|
+------+



In [ ]:
# Column operators
data=[(1,2,3),(4,5,6),(7,8,9)]
df=spark.createDataFrame(data).toDF("col1","col2","col3")
df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
|   7|   8|   9|
+----+----+----+



In [ ]:
df.select(df.col1 + df.col2).show()
df.select(df.col1 - df.col2).show() 
df.select(df.col1 * df.col2).show()
df.select(df.col1 / df.col2).show()
df.select(df.col1 % df.col2).show()
df.select(df.col2 > df.col3).show()
df.select(df.col2 < df.col3).show()
df.select(df.col2 == df.col3).show()

+-------------+
|(col1 + col2)|
+-------------+
|            3|
|            9|
|           15|
+-------------+

+-------------+
|(col1 - col2)|
+-------------+
|           -1|
|           -1|
|           -1|
+-------------+

+-------------+
|(col1 * col2)|
+-------------+
|            2|
|           20|
|           56|
+-------------+

+-------------+
|(col1 / col2)|
+-------------+
|          0.5|
|          0.8|
|        0.875|
+-------------+

+-------------+
|(col1 % col2)|
+-------------+
|            1|
|            4|
|            7|
+-------------+

+-------------+
|(col2 > col3)|
+-------------+
|        false|
|        false|
|        false|
+-------------+

+-------------+
|(col2 < col3)|
+-------------+
|         true|
|         true|
|         true|
+-------------+

+-------------+
|(col2 = col3)|
+-------------+
|        false|
|        false|
|        false|
+-------------+



In [ ]:
data=[(("James",None,"Smith"),"OH","M"),(("Anna","Rose",""),"NY","F"),(("Julia","","Williams"),"OH","F"),(("Maria","Anne","Jones"),"NY","M"),(("Jen","Mary","Brown"),"NY","M"),(("Mike","Mary","Williams"),"OH","M")]

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType        
schema = StructType([StructField('name', StructType([StructField('firstname', StringType(), True),StructField('middlename', StringType(), True),StructField('lastname', StringType(), True)])),StructField('state', StringType(), True),StructField('gender', StringType(), True)])
df2 = spark.createDataFrame(data = data, schema = schema)

In [ ]:
df2.printSchema()
df2.show(truncate=False) # shows all columns

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)

+----------------------+-----+------+
|name                  |state|gender|
+----------------------+-----+------+
|{James, null, Smith}  |OH   |M     |
|{Anna, Rose, }        |NY   |F     |
|{Julia, , Williams}   |OH   |F     |
|{Maria, Anne, Jones}  |NY   |M     |
|{Jen, Mary, Brown}    |NY   |M     |
|{Mike, Mary, Williams}|OH   |M     |
+----------------------+-----+------+



In [ ]:
df2.select("name.firstname","name.lastname").show(truncate=False)

+---------+--------+
|firstname|lastname|
+---------+--------+
|James    |Smith   |
|Anna     |        |
|Julia    |Williams|
|Maria    |Jones   |
|Jen      |Brown   |
|Mike     |Williams|
+---------+--------+



In [ ]:
df2.select("name.*").show(truncate=False)

+---------+----------+--------+
|firstname|middlename|lastname|
+---------+----------+--------+
|James    |null      |Smith   |
|Anna     |Rose      |        |
|Julia    |          |Williams|
|Maria    |Anne      |Jones   |
|Jen      |Mary      |Brown   |
|Mike     |Mary      |Williams|
+---------+----------+--------+

